In [4]:
#!/usr/bin/env python3
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import concurrent.futures
import pdb
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])
project = 'kfdrc-harmonization/sd-8y99qzjj'

## tag paired outputs

In [35]:
tasks = api.tasks.query(project=project, status="COMPLETED").all()
# task name search phrase
phrase = "KFDRC WES PRODUCTION: PNOC008-4"
# modify this to set which input file to use to tag the outputs with, may need to modify code if an array element
tumor_in_key = 'input_tumor_aligned'
normal_in_key = 'input_normal_aligned'
for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        metadata = task.inputs[tumor_in_key].metadata
        # del metadata['Kids First Biospecimen ID']
        metadata['Kids First Biospecimen ID Normal'] = task.inputs[normal_in_key].metadata['Kids First Biospecimen ID']
        metadata['Kids First Biospecimen ID Tumor'] = task.inputs[tumor_in_key].metadata['Kids First Biospecimen ID']
        for out_key in task.outputs:
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    for key in metadata:
                        file_obj.metadata[key] = metadata[key]
                    file_obj.save()
                else:
                    for output in task.outputs[out_key]:
                        file_obj = api.files.get(output.id)
                        for key in metadata:
                            file_obj.metadata[key] = metadata[key]
                        file_obj.save()
            except Exception as e:
                print(e)
                print("Skipping " + task.name + " due to error")

Valid task found KFDRC WES PRODUCTION: PNOC008-4 PT_5R8EWT55 BS_X0KN4VVW BS_CFJYJAA3


## tag single outputs

In [37]:
tasks = api.tasks.query(project=project, status="COMPLETED").all()
# task name search phrase
phrase = "kfdrc-consensus-calling run - PNOC008-4"
# modify this to set which input file to use to tag the outputs with, may need to modify code if an array element
in_key = 'lancet_vcf'
for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        metadata = task.inputs[in_key].metadata
        for out_key in task.outputs:
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    for key in metadata:
                        file_obj.metadata[key] = metadata[key]
                    file_obj.save()
                else:
                    for output in task.outputs[out_key]:
                        file_obj = api.files.get(output.id)
                        for key in metadata:
                            file_obj.metadata[key] = metadata[key]
                        file_obj.save()
            except Exception as e:
                print(e)
                print("Skipping " + task.name + " due to error")

Valid task found kfdrc-consensus-calling run - PNOC008-4


Requested file does not exist.
Skipping kfdrc-consensus-calling run - PNOC008-4 due to error


Valid task found kfdrc-consensus-calling run - PNOC008-4


## tag batch outputs

In [3]:
batch_id = 'ffa54e3d-31a8-47b1-b384-4fc61b0d9fcc'
batch_task = api.tasks.get(batch_id)
for task in batch_task.get_batch_children():
    # pdb.set_trace()
    sys.stderr.write('Valid task found ' + task.name + '\n')
    metadata = task.inputs['input_align'].metadata
    for out_key in task.outputs:
        try:
            file_obj = api.files.get(task.outputs[out_key].id)
            for key in metadata:
                file_obj.metadata[key] = metadata[key]
            file_obj.save()
        except Exception as e:
            print(e)
            print("Skipping " + task.name + " due to error")

Valid task found bcf-filter run - PNOC008-4: file: cf278813-3291-4f9b-b4f2-345936ed0796.cram
Valid task found bcf-filter run - PNOC008-4: file: BS_9TXEDSWB.Aligned.out.sorted.bam
Valid task found bcf-filter run - PNOC008-4: file: d5edd07c-2e3b-495a-99df-268e3d7411a6.cram


## update input file metadata

In [23]:
import time
manifest = open('/Users/brownm28/Documents/2019-May-7_PNOC008/pnoc008-4/mb_bam_in-manifest.tsv')
head = next(manifest)
header = head.rstrip('\n').split('\t')
tags = ['PNOC008-4']
for line in manifest:
    info = line.rstrip('\n').split('\t')
    file_obj = api.files.get(info[0])
    md = {}
    for i in range(3,len(info),1):
        if info[i] != "":
            md[header[i]] = info[i]
    file_obj.metadata = {}
    for key in md:
        file_obj.metadata[key] = md[key]
    file_obj.tags = tags
    try:
        file_obj.save()
        # time.sleep(5)
    except Exception as e:
        sys.stderr.write(str(e) + "\n")

## copy file meta data from file to others in manifest

In [5]:
file_id = '5dfcea21e4b054952cd36956'
file_obj = api.files.get(file_id)
metadata = {}
for key in file_obj.metadata:
    metadata[key] = file_obj.metadata[key]
manifest = open('/Users/brownm28/Documents/2019-May-7_PNOC008/pnoc008-4/to_add_meta-manifest.csv')
skip = next(manifest)
for line in manifest:
    info = line.split(',')
    cur = api.files.get(info[0])
    for key in metadata:
        cur.metadata[key] = metadata[key]
    cur.save()

## clear out old metadata - use with caution!

In [22]:
import time
flag = input()
if flag=="YAS":
    manifest = open('/Users/brownm28/Documents/2019-May-7_PNOC008/pnoc008-4/mb_bam_in-manifest.tsv')
    head = next(manifest)
    header = head.rstrip('\n').split('\t')
    tags = ['PNOC008-4']
    for line in manifest:
        info = line.rstrip('\n').split('\t')
        file_obj = api.files.get(info[0])
        file_obj.metadata = {}
        try:
            file_obj.save()
            # time.sleep(5)
        except Exception as e:
            sys.stderr.write(str(e) + "\n")

YAS


No relevant changes were detected in order to update the resource on the server.
No relevant changes were detected in order to update the resource on the server.
No relevant changes were detected in order to update the resource on the server.
No relevant changes were detected in order to update the resource on the server.
No relevant changes were detected in order to update the resource on the server.
No relevant changes were detected in order to update the resource on the server.


## Get nant api info

In [28]:
import requests
link = "https://five3.nantomics.com/api/clinical/"
token_file = open('/Users/brownm28/Documents/2019-May-7_PNOC008/pnoc008-4/nant_token')
token = next(token_file)
token = token.rstrip('\n')
response = requests.post(link, headers={'Authorization': 'access_token ' + token})
pdb.set_trace()
hold =1


--Return--
> <ipython-input-28-e58a243c52e3>(7)<module>()->None
-> pdb.set_trace()
(Pdb) p token
'aea983c63fe428a0b42fbcfa0f7a58cf60757742'
(Pdb) q


BdbQuit: 

## run task

In [33]:
old_task = api.tasks.get('79019fc3-13bc-4943-9d13-2575007edc7f')
in_dict = {}
tumor_bs_id = 'BS_X0KN4VVW'
normal_bs_id = 'BS_CFJYJAA3'
app_name = project + "/kfdrc-production-somatic-wes-variant-cnv-wf"
task_name = "KFDRC WES PRODUCTION: PNOC008-4 PT_5R8EWT55 " + tumor_bs_id + " " + normal_bs_id
for key in old_task.inputs:
    in_dict[key] = old_task.inputs[key]
in_dict['b_allele'] = api.files.get('5dfbf2efe4b054952cd31724')
in_dict['input_normal_name'] = normal_bs_id
in_dict['input_tumor_name'] = tumor_bs_id
in_dict['hg38_strelka_bed'] = api.files.get('5dfbe19fe4b09d9aaf41d466')
in_dict['input_normal_aligned'] = api.files.get('5dfac4f3e4b054952cd21099')
in_dict['input_tumor_aligned'] = api.files.get('5dfad46ee4b0e4c5064f6cd0')
in_dict['padded_capture_regions'] = api.files.get('5dfbe088e4b09d9aaf41d45a')
in_dict['unpadded_capture_regions'] = api.files.get('5dfbe0c8e4b09d9aaf41d45f')
in_dict['cnvkit_sex'] = 'Female'
in_dict['cfree_sex'] = 'XX'
task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
task.inputs['output_basename'] = task.id
task.save()


<Task: id=341ac65c-464c-4898-8453-752a4cfaadd4>